In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os
os.chdir('/content/drive/My Drive/ML Project/ML codes')
!ls

dataset			   LR_training.ipynb   MNB.ipynb
Dataset_Exploration.ipynb  LSTM_train.ipynb    models
dataset_preprocess.ipynb   ML_Pipelines.ipynb  outputs


In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('outputs/processed_ws.csv', encoding='latin-1')
df.head()

,text,label,hashtags,users,links,processed_text
0,"""like this if you are a tribe fan""",0,[],[],[],like this if you are a tribe fan
1,"""you're idiot.......................""",1,[],[],[],you re idiot
2,"""I am a woman Babs, and the only ""war on women...",1,[],[],[],i am a woman babs and the only war on women i ...
3,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",1,[],[],[],wow you benefitted so many wins this year from...
4,"""haha green me red you now loser whos winning ...",1,[],[],[],haha green me red you now loser whos winning n...


In [5]:
df['processed_text'] = df['processed_text'].str.cat(df['hashtags'], sep =' ')
df.head()

,text,label,hashtags,users,links,processed_text
0,"""like this if you are a tribe fan""",0,[],[],[],like this if you are a tribe fan []
1,"""you're idiot.......................""",1,[],[],[],you re idiot []
2,"""I am a woman Babs, and the only ""war on women...",1,[],[],[],i am a woman babs and the only war on women i ...
3,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",1,[],[],[],wow you benefitted so many wins this year from...
4,"""haha green me red you now loser whos winning ...",1,[],[],[],haha green me red you now loser whos winning n...


In [0]:
import re
def preprocess(text):
    text = re.sub(r'[^\w\d\s]', ' ', str(text))
    text = re.sub('[Ã]', ' ', str(text))
    return ' '.join(term for term in text.split())
df['processed_text'] = df.processed_text.apply(lambda row : preprocess(row))

In [7]:
df.head()

,text,label,hashtags,users,links,processed_text
0,"""like this if you are a tribe fan""",0,[],[],[],like this if you are a tribe fan
1,"""you're idiot.......................""",1,[],[],[],you re idiot
2,"""I am a woman Babs, and the only ""war on women...",1,[],[],[],i am a woman babs and the only war on women i ...
3,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",1,[],[],[],wow you benefitted so many wins this year from...
4,"""haha green me red you now loser whos winning ...",1,[],[],[],haha green me red you now loser whos winning n...


In [0]:
pd.DataFrame(df, columns=['text','label','hashtags', 'users', 'links', 'processed_text']).to_csv('outputs/train.csv', index = False)

In [0]:
tfidf_vec = TfidfVectorizer(ngram_range=(1,3), max_features=10000)
tfidf_text = tfidf_vec.fit_transform(df.processed_text)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_text, df['label'], test_size=0.2, random_state = 42)

classifier = MultinomialNB(alpha=0.2)
classifier.fit(X_train, y_train)

MultinomialNB(alpha=0.2, class_prior=None, fit_prior=True)

In [11]:
predictions = classifier.predict(X_test).tolist()
wrong = []
count = 0
for i in range(len(y_test)):
    if y_test.iloc[i] != predictions[i]:
        count += 1
        wrong.append(i)

      
print('Total number of test cases', len(y_test))
print('Number of wrong of predictions', count)


Total number of test cases 39278
Number of wrong of predictions 3324


In [12]:
from sklearn.metrics import classification_report
print(classification_report(predictions, y_test))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95     31548
           1       0.76      0.82      0.79      7730

    accuracy                           0.92     39278
   macro avg       0.86      0.88      0.87     39278
weighted avg       0.92      0.92      0.92     39278

